In [1]:
import numpy as np
from scipy.interpolate import Rbf
import plotly.plotly as py
import plotly.graph_objs as go
from mu2e.dataframeprod import DataFrameMaker
from mu2e.src.fiteval_c2 import FitFunctionMaker2
from interp_studies import interp_phi
from mu2e import mu2e_ext_path
import tqdm

In [2]:
df = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap',use_pickle = True).data_frame
ffm = FitFunctionMaker2('../mu2e/src/Mau10_800mm_long.csv')

In [3]:
#z = 6571
#y = 100
y = 100
z = 6521
xs = np.linspace(0,700,701)
buffer = 100 # for rbf
df_subset = df.query('{0}<=X<={1} and {2}<=Y<={3} and {4}<=Z<={5}'.format(
    xs[0]-100, xs[-1]+100, y-100, y+100, z-100, z+100))

In [4]:
bx_rbf = []
by_rbf = []
bz_rbf = []

bx_lacey = []
by_lacey = []
bz_lacey = []

bx_map = []
by_map = []
bz_map = []

for x in tqdm.tqdm(xs):
    # RBF mode
    df_tmp = df_subset.query('{0}<=X<={1}'.format(x-100,x+100))
    rbf_bx = Rbf(df_tmp.X, df_tmp.Y, df_tmp.Z, df_tmp.Bx, function='linear')
    rbf_by = Rbf(df_tmp.X, df_tmp.Y, df_tmp.Z, df_tmp.By, function='linear')
    rbf_bz = Rbf(df_tmp.X, df_tmp.Y, df_tmp.Z, df_tmp.Bz, function='linear')
    bx_rbf.append(float(rbf_bx(x,y,z)))
    by_rbf.append(float(rbf_by(x,y,z)))
    bz_rbf.append(float(rbf_bz(x,y,z)))
    
    # Lacey mode
    _, b_lacey = interp_phi(df_tmp, x, y, z, plot=False)
    bx_lacey.append(b_lacey[0])
    by_lacey.append(b_lacey[1])
    bz_lacey.append(b_lacey[2])
    
    # Field Map mode
    b_map = ffm.mag_field_function(x,y,z, True)
    bx_map.append(b_map[0])
    by_map.append(b_map[1])
    bz_map.append(b_map[2])

100%|██████████| 701/701 [00:51<00:00, 13.70it/s]


In [5]:
layout = dict(title = 'Bz v X, Y={0}, Z={1}'.format(y,z),
              xaxis = dict(title = 'X (mm)'),
              yaxis = dict(title = 'Bz (T)'),
              showlegend=True,
              )
trace_map = go.Scatter(
    x = xs,
    y = bz_map,
    name = 'fit map'
)
trace_rbf = go.Scatter(
    x = xs,
    y = bz_rbf,
    name = 'rbf'
)
trace_lacey = go.Scatter(
    x = xs,
    y = bz_lacey,
    name = 'lacey'
)
data = [trace_map, trace_rbf, trace_lacey]
fig = dict(data=data, layout=layout)
py.iplot(fig)

In [6]:
layout = dict(title = 'Bx v X, Y={0}, Z={1}'.format(y,z),
              xaxis = dict(title = 'X (mm)'),
              yaxis = dict(title = 'Bx (T)'),
              showlegend=True,
              )
trace_map = go.Scatter(
    x = xs,
    y = bx_map,
    name = 'fit map'
)
trace_rbf = go.Scatter(
    x = xs,
    y = bx_rbf,
    name = 'rbf'
)
trace_lacey = go.Scatter(
    x = xs,
    y = bx_lacey,
    name = 'lacey'
)
data = [trace_map, trace_rbf, trace_lacey]
fig = dict(data=data, layout=layout)
py.iplot(fig)

In [7]:
layout = dict(title = 'By v X, Y={0}, Z={1}'.format(y,z),
              xaxis = dict(title = 'X (mm)'),
              yaxis = dict(title = 'By (T)'),
              showlegend=True,
              )
trace_map = go.Scatter(
    x = xs,
    y = by_map,
    name = 'fit map'
)
trace_rbf = go.Scatter(
    x = xs,
    y = by_rbf,
    name = 'rbf'
)
trace_lacey = go.Scatter(
    x = xs,
    y = by_lacey,
    name = 'lacey'
)
data = [trace_map, trace_rbf, trace_lacey]
fig = dict(data=data, layout=layout)
py.iplot(fig)